<a href="https://colab.research.google.com/github/chrryyy/cs_4774_team_snorlax/blob/main/Analysis_of_Improving_Pedestrian_Safety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# read the dataset
crash_data = pd.read_csv('Crash_Data.csv')
crash_data.head()

,X,Y,OBJECTID,DOCUMENT_NBR,CRASH_YEAR,CRASH_DT,CRASH_MILITARY_TM,CRASH_SEVERITY,K_PEOPLE,A_PEOPLE,...,AREA_TYPE,SYSTEM,VSP,OWNERSHIP,PLAN_DISTRICT,MPO_NAME,RTE_NM,RNS_MP,NODE,OFFSET
0,-78.870556,38.645544,1,152915206,2015,2015/10/16 03:59:59+00,1700,A,0,2,...,Rural,VDOT Primary,2.0,1. State Hwy Agency,Central Shenandoah,NaN,R-VA SR00259SB,9.63,586049.0,834.0
1,-77.134973,38.943531,2,153060033,2015,2015/10/10 03:59:59+00,2350,O,0,0,...,Urban,NonVDOT secondary,7.0,4. Federal Roads,Northern Virginia,NOVA,R-VA OT90072NB,0.41,NaN,NaN
2,-79.499399,36.660702,3,153035144,2015,2015/10/29 03:59:59+00,1124,B,0,0,...,Rural,VDOT Secondary,6.0,1. State Hwy Agency,West Piedmont,NaN,R-VA071SC00750SB,5.90,518624.0,137.0
3,-77.336244,38.545770,4,150975078,2015,2015/04/06 03:59:59+00,900,O,0,0,...,Urban,VDOT Secondary,7.0,1. State Hwy Agency,Northern Virginia,NOVA,R-VA076SC00619EB,25.74,NaN,NaN
4,-79.337225,36.581073,5,152335159,2015,2015/08/21 03:59:59+00,751,O,0,0,...,Urban,NonVDOT primary,6.0,3. City or Town Hwy Agency,West Piedmont,DAN,R-VA US00058WB,301.27,519895.0,876.0


In [ ]:
# PED_NONPED